In [ ]:
!pip install youtube_transcript_api
!pip install transformers
!pip3 install git+https://github.com/ernie-mlg/rpunct.git
!pip install tqdm
import requests
import json
from youtube_transcript_api import YouTubeTranscriptApi
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ernie-mlg/rpunct.git to /tmp/pip-req-build-usqjiypd
  Running command git clone --filter=blob:none --quiet https://github.com/ernie-mlg/rpunct.git /tmp/pip-req-build-usqjiypd
  Resolved https://github.com/ernie-mlg/rpunct.git to commit 4c378cf570f05aa58c7d3dbb896b51ca2d95fde6
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
checkpoint = "sshleifer/distilbart-cnn-12-6"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
from rpunct import RestorePuncts
# The default language is 'english'
# rpunct = RestorePuncts(use_cuda=False)
rpunct = RestorePuncts()

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
%cd "/content/gdrive/MyDrive/IR_Project_W23"

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
/content/gdrive/.shortcut-targets-by-id/185mvSLpCVGSaXntfPSeAocQRS6XVVq-_/IR_Project_W23


In [ ]:
def fetch_most_replayed(video_id: str):
  r = requests.get(f'https://yt.lemnoslife.com/videos?part=mostReplayed&id={video_id}')
  if r.status_code != 200:
    raise Exception(f'Error while fetching most replayed for id: {video_id}')
  return json.loads(r.text)['items'][0]['mostReplayed']['heatMarkers']

In [ ]:
fetch_most_replayed('YieuqbIqUps')

[{'heatMarkerRenderer': {'timeRangeStartMillis': 0,
   'markerDurationMillis': 2060,
   'heatMarkerIntensityScoreNormalized': 1}},
 {'heatMarkerRenderer': {'timeRangeStartMillis': 2060,
   'markerDurationMillis': 2060,
   'heatMarkerIntensityScoreNormalized': 0.4254768856939025}},
 {'heatMarkerRenderer': {'timeRangeStartMillis': 4120,
   'markerDurationMillis': 2060,
   'heatMarkerIntensityScoreNormalized': 0.3844264537181312}},
 {'heatMarkerRenderer': {'timeRangeStartMillis': 6180,
   'markerDurationMillis': 2060,
   'heatMarkerIntensityScoreNormalized': 0.3299730038315575}},
 {'heatMarkerRenderer': {'timeRangeStartMillis': 8240,
   'markerDurationMillis': 2060,
   'heatMarkerIntensityScoreNormalized': 0.2926660463058963}},
 {'heatMarkerRenderer': {'timeRangeStartMillis': 10300,
   'markerDurationMillis': 2060,
   'heatMarkerIntensityScoreNormalized': 0.33168912891722424}},
 {'heatMarkerRenderer': {'timeRangeStartMillis': 12360,
   'markerDurationMillis': 2060,
   'heatMarkerIntensity

In [ ]:
def fetch_captions(video_id: str, languages=['en']):
  return YouTubeTranscriptApi.get_transcript(video_id, languages=languages)

In [ ]:
fetch_captions('YieuqbIqUps')

[{'text': 'sugar is okay for you know yes thank you',
  'start': 0.16,
  'duration': 6.8},
 {'text': 'how are you miss kulkarni how i am i am',
  'start': 3.28,
  'duration': 5.6},
 {'text': 'worried like all other mothers', 'start': 6.96, 'duration': 6.0},
 {'text': 'everywhere all the time we are worried',
  'start': 8.88,
  'duration': 8.08},
 {'text': 'your i am fine is my', 'start': 12.96, 'duration': 4.0},
 {'text': 'ass toy means toy gun', 'start': 29.119, 'duration': 5.28},
 {'text': 'it is not making any why we are doing do',
  'start': 32.48,
  'duration': 3.919},
 {'text': 'you have to stop this', 'start': 34.399, 'duration': 4.641},
 {'text': 'see last year my papa gave pratibha',
  'start': 36.399,
  'duration': 4.16},
 {'text': 'kitchen set', 'start': 39.04, 'duration': 4.08},
 {'text': 'my husband got so upset just because',
  'start': 40.559,
  'duration': 4.401},
 {'text': "she's girl you are giving her kitchen",
  'start': 43.12,
  'duration': 2.72},
 {'text': 'set', 

In [ ]:
def remove_caption_overlaps(captions: list):
  for i in range(len(captions) - 1):
    captions[i]['duration'] = min(captions[i]['duration'], captions[i + 1]['start'] - captions[i]['start'])
  return captions

In [ ]:
remove_caption_overlaps(fetch_captions('YieuqbIqUps'))

[{'text': 'sugar is okay for you know yes thank you',
  'start': 0.16,
  'duration': 3.1199999999999997},
 {'text': 'how are you miss kulkarni how i am i am',
  'start': 3.28,
  'duration': 3.68},
 {'text': 'worried like all other mothers',
  'start': 6.96,
  'duration': 1.9200000000000008},
 {'text': 'everywhere all the time we are worried',
  'start': 8.88,
  'duration': 4.08},
 {'text': 'your i am fine is my', 'start': 12.96, 'duration': 4.0},
 {'text': 'ass toy means toy gun',
  'start': 29.119,
  'duration': 3.360999999999997},
 {'text': 'it is not making any why we are doing do',
  'start': 32.48,
  'duration': 1.919000000000004},
 {'text': 'you have to stop this', 'start': 34.399, 'duration': 2.0},
 {'text': 'see last year my papa gave pratibha',
  'start': 36.399,
  'duration': 2.6409999999999982},
 {'text': 'kitchen set', 'start': 39.04, 'duration': 1.5189999999999984},
 {'text': 'my husband got so upset just because',
  'start': 40.559,
  'duration': 2.561},
 {'text': "she's 

In [ ]:
def fuse_captions_most_replayed(captions: list, most_replayed: list):
  # ind for iterating over most_replayed
  ind = 0
  n = len(most_replayed)

  for caption in captions:
    end_dur = caption['start'] + caption['duration']
    intensity = 0
    count = 0

    while ind < n and most_replayed[ind]['heatMarkerRenderer']['timeRangeStartMillis'] / 1000 < caption['start']:
      ind += 1
    
    while ind < n and most_replayed[ind]['heatMarkerRenderer']['timeRangeStartMillis'] / 1000 < end_dur:
      intensity += most_replayed[ind]['heatMarkerRenderer']['heatMarkerIntensityScoreNormalized']
      count += 1
      ind += 1
    
    if count:
      intensity = intensity / count

    caption['intensity'] = intensity
  return captions

In [ ]:
def get_fused_captions_intensity(video_id: str):
  most_replayed = fetch_most_replayed(video_id)
  captions = fetch_captions(video_id)
  remove_caption_overlaps(captions)
  return fuse_captions_most_replayed(captions, most_replayed)

In [ ]:
get_fused_captions_intensity('YieuqbIqUps')

[{'text': 'sugar is okay for you know yes thank you',
  'start': 0.16,
  'duration': 3.1199999999999997,
  'intensity': 0.4254768856939025},
 {'text': 'how are you miss kulkarni how i am i am',
  'start': 3.28,
  'duration': 3.68,
  'intensity': 0.35719972877484435},
 {'text': 'worried like all other mothers',
  'start': 6.96,
  'duration': 1.9200000000000008,
  'intensity': 0.2926660463058963},
 {'text': 'everywhere all the time we are worried',
  'start': 8.88,
  'duration': 4.08,
  'intensity': 0.3355336270705944},
 {'text': 'your i am fine is my',
  'start': 12.96,
  'duration': 4.0,
  'intensity': 0.3564911949914417},
 {'text': 'ass toy means toy gun',
  'start': 29.119,
  'duration': 3.360999999999997,
  'intensity': 0.17579769399583717},
 {'text': 'it is not making any why we are doing do',
  'start': 32.48,
  'duration': 1.919000000000004,
  'intensity': 0.14911336176971499},
 {'text': 'you have to stop this',
  'start': 34.399,
  'duration': 2.0,
  'intensity': 0.1206053114242

In [ ]:
import nltk
nltk.download('punkt')
def str_to_sentences(string_data: str):
  return nltk.tokenize.sent_tokenize(string_data)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def create_chunks(sentences: list):
  # initialize
  length = 0
  chunk = ""
  chunks = []
  count = -1
  for sentence in sentences:
    count += 1
    combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

    if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
      chunk += sentence + " " # add the sentence to the chunk
      length = combined_length # update the length counter

      # if it is the last sentence
      if count == len(sentences) - 1:
        chunks.append(chunk.strip()) # save the chunk
      
    else: 
      chunks.append(chunk.strip()) # save the chunk
      
      # reset 
      length = 0 
      chunk = ""

      # take care of the overflow sentence
      chunk += sentence + " "
      length = len(tokenizer.tokenize(sentence))
  return chunks

In [ ]:
def summary(chunks):
  inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]
  outputs = ''
  for input in inputs:
    output = model.generate(**input)
    outputs += tokenizer.decode(*output, skip_special_tokens=True)
  return outputs

In [ ]:
def get_summary(long_string: str):
  sents = str_to_sentences(long_string)
  chunks = create_chunks(sents)
  summ = summary(chunks)
  return summ

In [ ]:
def process_video_list(video_ids: list, filepath: str):
  result = []
  for id in tqdm(video_ids):
    try:
      fused_captions = get_fused_captions_intensity(id)
      text_captions = ' '.join([caption['text'] for caption in fused_captions])
      ARTICLE = rpunct.punctuate(text_captions)
      summary = get_summary(ARTICLE)

    except:
      continue

    result.append({
        'id': id,
        'text_captions': ARTICLE,
        'fused_captions': fused_captions,
        'summary': summary
    })

  # Serializing json
  json_object = json.dumps(result, indent=4)
  
  # Writing to sample.json
  with open(filepath, "w") as outfile:
      outfile.write(json_object)

In [ ]:
cnn10_video_ids = [
    '8GgaCgjAFLM',
    'Kc_GMYHJNFQ',
    'wPdiuj2JwEU',
    'H07TJT3VpSY',
    'U4WxR7hXfCQ',
    '1xbEbTJASoE',
    'rQp4putiL_A',
    'rQp4putiL_A',
    '94N_3jBAU_c',
    'owDr-ra0P6I'
]
# process_video_list(cnn10_video_ids, 'data/cnn10.json')

In [ ]:
def fetch_channelID_from_username(username: str):
  r = requests.get(f'https://yt.lemnoslife.com/noKey/channels?forUsername={username}&part=id')
  if r.status_code != 200 or 'items' not in json.loads(r.text).keys():
    raise Exception(f'Error while fetching channel ID for username: {username}')
  return json.loads(r.text)['items'][0]['id']

In [ ]:
def fetch_videos_for_channelID(channelID: str, maxResults: int):
  ans = []
  pageToken = ''
  while(len(ans) < maxResults):
    link = f'https://yt.lemnoslife.com/noKey/search?channelId={channelID}&part=id&order=date&maxResults={maxResults}'
    if pageToken:
      link += f'&pageToken={pageToken}'
    r = requests.get(link)
    if r.status_code != 200 or 'items' not in json.loads(r.text).keys():
      raise Exception(f'Error while fetching videoes for channelID: {channelID}')
    
    json_obj = json.loads(r.text)
    
    if len(json_obj['items']) == 0:
      break

    for item in json_obj['items']:
      if 'videoId' in item['id'].keys():
        ans.append(item['id']['videoId'])

    if 'nextPageToken' in json_obj.keys():
      pageToken = json_obj['nextPageToken']
    else:
      break
  return ans

In [ ]:
c_id = fetch_channelID_from_username('dhruvrathee')
fetch_videos_for_channelID(c_id, 1000)

['p9ju0x3xVCA',
 'D-iPbtcxgM8',
 'wvhhuZWwZnA',
 'sDEZSFNQmP0',
 '-iDMd0JHYXc',
 'Y-k2eaDcyqo',
 'zMZ6IBCQCm8',
 'CFZWI8ExfuY',
 'vb1GIHNY4OE',
 'L1UC_bnCk8k',
 'Hu-Bdubnnj0',
 'ueUyKgO079I',
 '7RuNk6L9ekk',
 '3cvag-9tCW0',
 '1unoB93xeeQ',
 'R1gTM30ZRkM',
 'c1YmChaP01g',
 'NoxkNvz6Dqo',
 'DUel5IzusKc',
 'JILyj5IZ3rI',
 'byC5e6X9q80',
 'j0lm6JIfD38',
 'hVvAkPbJwnw',
 'mTrsShdSNKg',
 '8VKnGdl4Q3U',
 'MJ_vmdIJIRw',
 'CFGG-mFZBHo',
 'Vh6n-0B29aw',
 'YrEwBEqYohM',
 '85SAO5HYrjc',
 'Aesag8J7EeM',
 '1fuau2Idip8',
 'YcMEQtxBH6k',
 'y_isRZIrAUQ',
 'Hmhp1ZYPC5w',
 'vXVRs_Sar6Y',
 '5iy4p_bYu5Q',
 'TTGczb8uLA0',
 'RPiXj1t4wRQ',
 'bJHEPmu837Y',
 'q0PbcaBgmWk',
 '1CR3hS5Y-PI',
 'X50iFxhUWiU',
 'xIEzhdj5G5k',
 '3bZoB8PiXas',
 'Lr7GMr1NzV0',
 'mX3ZWEG9lyw',
 '5No-IWBTBaQ',
 'qlMBNrY7ABo',
 'd0KuhikVnVs',
 'JE22hiUR2dI',
 'oqC5mg2k4Tk',
 'b8n-wpvBjrw',
 'JWgwHjfhA3A',
 't_cSk9tQQkk',
 '85nMXO_nRxc',
 'Ih_4C6DJ0EU',
 'ji1fQRWJqlc',
 'vh9rBp_4eHU',
 'HvWlcpf48x4',
 'TGHqBX1YJRc',
 'wQhiQrt2TqQ',
 'w6l4Gs

In [ ]:
def fetch_videos_for_playlistID(playlistId: str, maxResults: int):
  ans = []
  pageToken = ''
  while(len(ans) < maxResults):
    link = f'https://yt.lemnoslife.com/playlistItems?part=snippet&playlistId={playlistId}&maxResults={min(maxResults, maxResults - len(ans))}'
    if pageToken:
      link += f'&pageToken={pageToken}'
    r = requests.get(link)
    if r.status_code != 200 or 'items' not in json.loads(r.text).keys():
      raise Exception(f'Error while fetching videoes for playlistId: {playlistId}')
    
    json_obj = json.loads(r.text)
    
    if len(json_obj['items']) == 0:
      break

    for item in json_obj['items']:
      if 'videoId' in item['snippet']['resourceId'].keys():
        ans.append(item['snippet']['resourceId']['videoId'])

    if 'nextPageToken' in json_obj.keys():
      pageToken = json_obj['nextPageToken']
    else:
      break
  return ans

In [ ]:
fetch_videos_for_playlistID('PL9WYcwsWaJ7owjsIjNDQwdfQ8ZBNRbsTi', 100)

['OTRnnUPcZCE',
 'imNgPIeRE5Y',
 'dqRsO3HPpQw',
 '1ffXBrZnbyw',
 'flurYbLRGbc',
 '4MyK2bL3Yvo',
 'QiG49Cj5BFE',
 '8uw9CGDTXAo',
 '9YP1UmX1si4',
 'vNh2bW4rdO0',
 'mnupD3aQPss',
 '4apaSV0OWNQ',
 'mVEqRt3ctvU',
 'WJ4eOYP_Fy8',
 'SCGPP68PUdk',
 'Gjoxxg0c4MA',
 'DtkUulIWjH0',
 'bLUDX4mPiro',
 '-r3aclHSNsw']

## Preparing CNN dataset

In [ ]:
cnn_videos_list = fetch_videos_for_channelID('UCTOoRgpHTjAQPk6Ak70u-pA', 100)
process_video_list(cnn_videos_list, 'data/cnn10_full.json')

## Preparing TED playlist dataset

In [ ]:
cnn_videos_list = fetch_videos_for_playlistID('PLlT0ph_Ig5Rc8xVBw47aZfLsOGw-lxRWb', 125)
process_video_list(cnn_videos_list[:125], 'data/ted.json')

100%|██████████| 125/125 [2:15:44<00:00, 65.15s/it]


## Preparing Cooking playlist dataset

In [ ]:
ramsay_videos_list = fetch_videos_for_playlistID('PLTzMGnJjrsSy6H8uiX3XKWtUhhj2Vilw1', 100)
process_video_list(ramsay_videos_list[:100], 'data/ramsay10.json')

100%|██████████| 31/31 [18:34<00:00, 35.95s/it]


## Preparing Tech Reviews playlist dataset

In [ ]:
reviews_videos_list = fetch_videos_for_playlistID('PLBsP89CPrMeNm71T5gYC6jebm9vPbLBiP', 125)
process_video_list(reviews_videos_list[:125], 'data/reviews.json')

  0%|          | 0/125 [00:00<?, ?it/s]/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (142) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 125/125 [1:51:12<00:00, 53.38s/it]


## Preapring podcasts dataset

In [ ]:
reviews_videos_list = fetch_videos_for_playlistID('PL7MFKrRPjBsseGrwSpGX0EkFNna7EzvZi', 150)
process_video_list(reviews_videos_list[:150], 'data/podcasts.json')

  0%|          | 0/146 [00:00<?, ?it/s]/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (142) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 146/146 [1:32:48<00:00, 38.14s/it]
